In [ ]:
def register_update_tabs_callback(app, name):
    @app.callback(
        Output('file_tabs' + name, 'tabs'),
        Output('file_tabs' + name, 'children'),
        Input('uploaded-file' + name, 'data'),
        Input('file_tabs' + name, 'active_tab'),
    )
    def update_tabs(data, active_tab):
        """
        We build dynamic tabs based on the files. For each tab, we show
        an <iframe> (or <object>) to display the PDF or text.
        """
        if not data or not data[0]:
            return [], []

        encoded_files, file_texts, file_names = data
        num_files = len(encoded_files)

        # Build a list of tab definitions
        tabs = [
            {'id': f'tab{i + 1}{name}', 'title': file_names[i]}
            for i in range(num_files)
        ]

        # If no active tab, default to first tab
        if not active_tab:
            active_tab = f'tab1{name}'

        # Get the index of the active tab
        try:
            # Extract the number from the tab ID (e.g., 'tab2AWM' -> 2)
            tab_num = int(''.join(filter(str.isdigit, active_tab)))
            tab_id = tab_num - 1  # Convert to 0-based index
        except (ValueError, TypeError):
            tab_id = 0

        # Ensure tab_id is valid
        tab_id = max(0, min(tab_id, len(encoded_files) - 1))

        # Create iframe for the active tab
        children = [
            html.Div([
                html.Iframe(
                    id=f'pdf-viewer-{tab_id}{name}',
                    src=encoded_files[tab_id],
                    style={
                        'width': '100%',
                        'height': '800px',
                        'border': 'none',
                        'marginTop': '10px'
                    }
                )
            ], style={
                'width': '100%',
                'height': '850px',
                'padding': '20px',
                'backgroundColor': '#ffffff'
            })
        ]

        return tabs, children

In [ ]:
def register_file_upload_callback(app, name):
    @app.callback(
        Output('uploaded-file' + name, 'data'),
        Input('upload-data' + name, 'contents'),
        State('upload-data' + name, 'filename'),
        State('uploaded-file' + name, 'data')
    )
    def _upload_files(contents, filenames, current_data):
        if not contents:
            return current_data if current_data else ([], [], [])

        encoded_files = current_data[0] if current_data else []
        file_texts = current_data[1] if current_data else []
        file_names = current_data[2] if current_data else []

        # Convert to lists if single file
        if not isinstance(contents, list):
            contents = [contents]
            filenames = [filenames]

        for content, filename in zip(contents, filenames):
            try:
                # Ensure proper content formatting
                if not content.startswith('data:'):
                    print(f"Warning: Content for {filename} missing data URI prefix")
                    continue

                ext = filename.lower().split('.')[-1]
                
                if ext == 'pdf':
                    # Ensure proper PDF mime type
                    if 'application/pdf' not in content:
                        # Fix the content type if needed
                        _, b64_content = content.split('base64,', 1)
                        content = f'data:application/pdf;base64,{b64_content}'
                    
                    encoded_files.append(content)
                    file_texts.append("")  # PDF text is not needed for display
                    file_names.append(filename)
                    print(f"Successfully processed PDF: {filename}")
                
                elif ext == 'txt':
                    encoded_files.append(content)
                    try:
                        _, b64_str = content.split('base64,', 1)
                        text = base64.b64decode(b64_str).decode('utf-8')
                        file_texts.append(text)
                    except Exception as e:
                        print(f"Error decoding text: {e}")
                        file_texts.append("")
                    file_names.append(filename)

            except Exception as e:
                print(f"Error processing {filename}: {e}")
                continue

        return encoded_files, file_texts, file_names